# Model Definition and Evaluation
## Table of Contents
1. [Model Selection](#model-selection)
2. [Feature Engineering](#feature-engineering)
3. [Hyperparameter Tuning](#hyperparameter-tuning)
4. [Implementation](#implementation)
5. [Evaluation Metrics](#evaluation-metrics)
6. [Comparative Analysis](#comparative-analysis)


In [ ]:
import sys

import numpy as np
import tensorflow as tf
import tf_keras
import transformers

# Add the parent directory to the path so that the custom utilities module is found
sys.path.insert(0, "..")

import utilities

## Model Selection

The first step was to try an LSTM model to see if the preserving of sequence improves model performance. The LSTM model performed similar to the baseline model. Next, I tried using pre-trained embeddings to see if leveraging models trained on larger datasets aids performance. However, model performance did not increase, but even declined when using pre-trained embeddings from a Neural-Net Language Model. Lastly, a more advanced Transformer-based model was fine tuned to the project task, which finally yielded a substantial improvement over baseline performance.

## Feature Engineering

No additional feature engineering was done compared to the data preparation already implemented for the baseline model. However, for the Transformer-based model, stop words were left in, and larger sequence lengths were used.

In [ ]:
has_duplicate_removal = True
has_stop_word_removal = True
validation_size = 0.1
test_size = 0.2

x_training, x_validation, x_test, y_training, y_validation, y_test = (
    utilities.prepare_and_split_data(
        has_duplicate_removal=has_duplicate_removal,
        has_stop_word_removal=has_stop_word_removal,
        validation_size=validation_size,
        test_size=test_size,
    )
)

print(f"Training set size: {len(x_training)}")
print(f"Validation set size: {len(x_validation)}")
print(f"Test set size: {len(x_test)}")

## Hyperparameter Tuning

Hyperparameters were tuned manually by trying different parameters such as layer size and learning rates and seeing what works best. In general, model performance was similar for varying parameter values. Naturally, automated hyperparameter tuning would be preferable over manual tuning, as this could systematically cover a wider range of possible values and objectively find an optimal solution. Such an automated approach using Grid Search or Random Search may be implemented in future versions.

## Implementation

As stated above, three different models were implemented and compared to the baseline model.

### Long Short-Term Memory Model

The first model is a bidirectional LSTM-model, which preservence information about the sequence of the words/tokens. As with the baseline model, embeddings are trained from scratch.

In [ ]:
vocabulary_size = 10_000
maximum_sequence_length = 12
standardization_method = "lower"
dimensions_embedding = 32
units_lstm_1 = 64
units_lstm_2 = 32
units_lstm_3 = 8
units_dense = 8

model_lstm = utilities.build_model_lstm(
    x_training=x_training,
    vocabulary_size=vocabulary_size,
    maximum_sequence_length=maximum_sequence_length,
    standardization_method=standardization_method,
    dimensions_embedding=dimensions_embedding,
    units_lstm_1=units_lstm_1,
    units_lstm_2=units_lstm_2,
    units_lstm_3=units_lstm_3,
    units_dense=units_dense,
)

model_lstm.summary()

In [ ]:
number_of_epochs = 5
batch_size = 512

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.BinaryCrossentropy()
metrics = [tf.keras.metrics.BinaryAccuracy()]

model_lstm.compile(optimizer=optimizer, loss=loss, metrics=metrics)

history_lstm = model_lstm.fit(
    x_training,
    y_training,
    epochs=number_of_epochs,
    batch_size=batch_size,
    validation_data=(x_validation, y_validation),
)

In [ ]:
y_limits = (0.75, 1)

figure, axes = utilities.plot_accuracy(history=history_lstm, y_limits=y_limits)

In [ ]:
figure, axes = utilities.plot_loss(history=history_lstm)

### Model With Pre-Trained Embeddings From a Neural-Net Language Model

The second model is a structurally similar to the baseline model, but instead of training embeddings from scratch, pre-trained embeddings from an NNLM are used.

In [ ]:
embedding_dimensions = 128  # can be 50 or 128
units_dense = 16
is_trainable = True

model_nnlm = utilities.build_model_nnlm(
    embedding_dimensions=embedding_dimensions,
    units_dense=units_dense,
)

model_nnlm.summary()

In [ ]:
number_of_epochs = 50
batch_size = 512

initial_learning_rate = 5e-5

# tf_keras is used instead of tf.keras, because tensorflow_hub (Version 0.16.1) is incompatible with Keras 3
optimizer = tf_keras.optimizers.Adam(learning_rate=initial_learning_rate)
loss = tf_keras.losses.BinaryCrossentropy()
metrics = [tf_keras.metrics.BinaryAccuracy()]

model_nnlm.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics,
)

history_nnlm = model_nnlm.fit(
    x_training,
    y_training,
    epochs=number_of_epochs,
    batch_size=batch_size,
    validation_data=(x_validation, y_validation),
)

In [ ]:
y_limits = (0.5, 1)
x_ticks = np.arange(start=5, stop=51, step=5).tolist()

figure, axes = utilities.plot_accuracy(history=history_nnlm, y_limits=y_limits, x_ticks=x_ticks)

In [ ]:
figure, axes = utilities.plot_loss(history=history_nnlm, x_ticks=x_ticks)

### Fine-Tuned German BERT

The third and final model is a fine-tuned version of the Transformer-based German BERT model.

In [ ]:
has_duplicate_removal = True
has_stop_word_removal = False

x_training, x_validation, x_test, y_training, y_validation, y_test = (
    utilities.prepare_and_split_data(
        has_duplicate_removal=has_duplicate_removal,
        has_stop_word_removal=has_stop_word_removal,
        validation_size=validation_size,
        test_size=test_size,
    )
)

In [ ]:
model_name_gbert = "deepset/gbert-base"
maximum_sequence_length = 30

data_training = utilities.tokenize_text_gbert(
    texts=x_training.tolist(),
    labels=y_training.tolist(),
    maximum_sequence_length=maximum_sequence_length,
    model_name=model_name_gbert,
)

data_validation = utilities.tokenize_text_gbert(
    texts=x_validation.tolist(),
    labels=y_validation.tolist(),
    maximum_sequence_length=maximum_sequence_length,
    model_name=model_name_gbert,
)

data_test = utilities.tokenize_text_gbert(
    texts=x_test.tolist(),
    labels=y_test.tolist(),
    maximum_sequence_length=maximum_sequence_length,
    model_name=model_name_gbert,
)

model_gbert = transformers.TFBertForSequenceClassification.from_pretrained(
    model_name_gbert,
    num_labels=1,
)

model_gbert.summary()

In [ ]:
epochs = 5
batch_size = 64
initial_learning_rate = 5e-5

# tf_keras is used instead of tf.keras, because transformers (Version 4.43.3) is incompatible with Keras 3
optimizer = tf_keras.optimizers.Adam(learning_rate=initial_learning_rate)
loss = tf_keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [tf_keras.metrics.BinaryAccuracy()]

model_gbert.compile(optimizer=optimizer, loss=loss, metrics=metrics)

history_gbert = model_gbert.fit(
    data_training.shuffle(1000).batch(batch_size),
    validation_data=data_validation.batch(batch_size),
    epochs=epochs,
    batch_size=batch_size,
)

In [ ]:
y_limits = (0.9, 1)

figure, axes = utilities.plot_accuracy(history=history_gbert, y_limits=y_limits)

In [ ]:
figure, axes = utilities.plot_loss(history=history_gbert)

## Evaluation Metrics

As with the baseline model, accuracy was chosen as the evaluation metric, since the two classes are well-balanced.

In [ ]:
accuracy_lstm = utilities.evaluate_model(
    model=model_lstm,
    x_test=x_test,
    y_test=y_test,
)

accuracy_nnlm = utilities.evaluate_model(
    model=model_nnlm,
    x_test=x_test,
    y_test=y_test,
)

accuracy_gbert = utilities.evaluate_model(
    model=model_gbert,
    x_test=data_test.batch(batch_size=batch_size),
    y_test=y_test,
)

## Comparative Analysis

Performance was already quite high for the baseline model. This shows that a fairly simple model can already solve the task at hand pretty well, maybe because distinguishing between plain and regular language is not a particularly difficult task. The LSTM model performs similarly to the baseline model, suggesting that preserving word order does not improve model performance. Surprisingly, the model with pre-trained NNLM-embeddings performed even worse than the simple baseline model. The reasons for this bad performance are not quite clear. FInally, the fine-tuned version of German BERT outperformed all other models by a substantial margin. This high performance shows that the more advanced Transormer-based approach yields a superior text-classification accuracy, which is in line with existing research on the topic.

In [ ]:
accuracy_baseline = 0.876

print(f"Accuracy Baseline: {accuracy_baseline * 100:.1f}%")
print(f"Accuracy LSTM:  {accuracy_lstm * 100:.1f}%")
print(f"Accuracy NNLM: {accuracy_nnlm * 100:.1f}%")
print(f"Accuracy German BERT: {accuracy_gbert * 100:.1f}%")